## dataset denver download if you download than pass

In [5]:
import deepmimo as dm

# Search for scenarios matching criteria
scenarios = dm.search()

In [6]:
scenarios

['asu_campus_3p5',
 'city_0_newyork_3p5',
 'city_1_losangeles_3p5',
 'city_2_chicago_3p5',
 'city_3_houston_3p5',
 'city_4_phoenix_3p5',
 'city_5_philadelphia_3p5',
 'city_6_miami_3p5',
 'city_7_sandiego_3p5',
 'city_8_dallas_3p5',
 'city_9_sanfrancisco_3p5',
 'city_10_austin_3p5',
 'city_11_santaclara_3p5',
 'city_12_fortworth_3p5',
 'city_13_columbus_3p5',
 'city_14_charlotte_3p5',
 'city_15_indianapolis_3p5',
 'city_16_sanfrancisco_3p5',
 'city_17_seattle_3p5',
 'city_18_denver_3p5',
 'city_19_oklahoma_3p5',
 'i1_2p5',
 'i2_28b',
 'city_0_newyork_28',
 'city_1_losangeles_28',
 'city_2_chicago_28',
 'city_3_houston_28',
 'city_4_phoenix_28',
 'city_5_philadelphia_28',
 'city_7_sandiego_28',
 'city_6_miami_28',
 'city_8_dallas_28',
 'city_9_sanfrancisco_28',
 'city_10_austin_28',
 'city_11_santaclara_28',
 'city_12_fortworth_28',
 'city_13_columbus_28',
 'city_14_charlotte_28',
 'city_15_indianapolis_28',
 'city_16_sanfrancisco_28',
 'city_17_seattle_28',
 'city_18_denver_28',
 'city_

In [1]:
import deepmimo as dm

# Download a specific scenario
dm.download('city_18_denver_28')

Scenario "city_18_denver_28" already exists in /home/dlghdbs200/LWM_denver/deepmimo_scenarios


# dataset load

In [2]:
import deepmimo as dm

# Load dataset
dataset = dm.load("city_18_denver_28")

# Access dataset properties
aoa_az = dataset.aoa_az
aoa_el = dataset.aoa_el
inter_pos = dataset.inter_pos

# Compute specific channel information
# los = dataset.compute_los()
channels = dataset.compute_channels()
pl = dataset.compute_pathloss()

Loading TXRX PAIR: TXset 1 (tx_idx 0) & RXset 0 (rx_idxs 43248)
Loading TXRX PAIR: TXset 2 (tx_idx 0) & RXset 0 (rx_idxs 43248)
Loading TXRX PAIR: TXset 3 (tx_idx 0) & RXset 0 (rx_idxs 43248)


Generating channels: 100%|█████████████████████████████████████████████████████| 43248/43248 [00:01<00:00, 29975.82it/s]


# Datashape
deepmimo/generator/dataset.py def compute_channels in here channel data shape

            numpy.ndarray: MIMO channel matrix with shape [n_users, n_rx_ant, n_tx_ant, n_subcarriers]
                          if freq_domain=True, otherwise [n_users, n_rx_ant, n_tx_ant, n_paths]

# three TxSet(1,2,3) 
“For 43,248 users, 1 user antenna, 8 BS antennas, and 1 propagation path.”
(users, user antenna, BS antennas, PL)

In [5]:
channels[1].shape

(43248, 1, 8, 1)

In [3]:
channels[0].shape

(43248, 1, 8, 1)

In [31]:
channels[1][0][0]

array([[-3.0468159e-10-5.8945265e-10j],
       [ 2.3423355e-10+4.7612841e-10j],
       [-2.8508820e-10-3.6341480e-10j],
       [ 4.1828480e-10+3.4321815e-10j],
       [-5.3884103e-10-4.4782919e-10j],
       [ 5.5945587e-10+6.2805278e-10j],
       [-4.5894541e-10-7.8667256e-10j],
       [ 2.9558433e-10+8.4439894e-10j]], dtype=complex64)

In [67]:
pip list

Package                   Version
------------------------- --------------
ace_tools                 0.0
anyio                     4.9.0
argon2-cffi               25.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
Automat                   20.2.0
babel                     2.17.0
bcrypt                    3.2.0
beautifulsoup4            4.13.4
bleach                    6.2.0
blinker                   1.4
certifi                   2020.6.20
cffi                      1.17.1
chardet                   4.0.0
charset-normalizer        3.4.2
click                     8.0.3
cloud-init                25.1.4
colorama                  0.4.4
comm                      0.2.2
command-not-found         0.3
configobj                 5.0.6
constantly                15.1.0
contourpy                 1.3.2
cpplint                   1.5.5
cryptography              3.4.8
cycler                

# data preprocessing

## How to predict 
H1,H2,H3 -> all data union

In [46]:
H1, H2, H3 = channels[0], channels[1], channels[2]
X = np.concatenate([H1,H2,H3], axis = 0)
y = X.copy()

# 1) Numpy -> Tensor (+ float32) & (optional) per-sample normalization
X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(y).float()

eps = 1e-8
# L2 norm 
scale = torch.linalg.vector_norm(X_t.view(X_t.size(0), -1), dim=1, keepdim=True).clamp_min(eps)
X_t = X_t / scale.view(-1,1,1,1)
y_t = y_t / scale.view(-1,1,1,1)

# 2) Split only by users (inputs=H1,H2; label=H3)
dataset = TensorDataset(X_t, y_t)
N = len(dataset)
n_tr = int(N*0.75)         # e.g., 75% train, 25% val
n_val = N - n_tr

# random seed 42 fixed
train_set, val_set = random_split(dataset, [n_tr, n_val], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_set, batch_size=1024, shuffle=True)
val_loader   = DataLoader(val_set,   batch_size=2048, shuffle=False)

print(f"Train samples: {len(train_set)}, Val samples: {len(val_set)}")


Train samples: 97308, Val samples: 32436


In [43]:
X.shape

(86496, 1, 8, 1)

In [39]:
y.shape

(43248, 1, 8, 1)

# Model define

In [52]:
# 3) Models (same interface as your run())
class MLP(nn.Module):
    def __init__(self, hidden=256, depth=3, pdrop=0.1):
        super().__init__()
        layers, d = [], 8
        for _ in range(depth-1):
            layers += [nn.Linear(d, hidden), nn.ReLU(), nn.Dropout(pdrop)]
            d = hidden
        layers += [nn.Linear(d, 8)]
        self.net = nn.Sequential(*layers)
    def forward(self, x):                 # x: (B,1,16,1)
        x = x.view(x.size(0), -1)         # (B,16)
        y = self.net(x)                   # (B,8)
        return y.view(-1,1,8,1)           # (B,1,8,1)

class CNN1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Conv1d(1, 32, 3, padding=1), nn.ReLU(),
            nn.Conv1d(32, 64, 3, padding=1), nn.ReLU(),
            nn.Conv1d(64, 64, 3, padding=1), nn.ReLU(),
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),  # (B,64,16) -> (B,64,1)
            nn.Flatten(),             # (B,64)
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 8)
        )
    def forward(self, x):                 # x: (B,1,16,1)
        x = x.squeeze(-1)                 # (B,1,16)
        h = self.feat(x)                  # (B,64,16)
        y = self.head(h)                  # (B,8)
        return y.view(-1,1,8,1)



# Evaluate

In [63]:
def run(model, epochs=150, lr=1e-3, weight_decay=0.0, show_every=1):
    model = model.to(device)
    criterion = nn.MSELoss(reduction="mean")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_loss = float("inf")
    best_nmse = float("inf")
    best_rmse = float("inf")
    best_nmse_db = float("inf")

    best_epoch_loss = 0
    best_epoch_nmse = 0
    best_epoch_rmse = 0
    best_epoch_nmse_db = 0

    for ep in range(1, epochs + 1):
        t0 = time.time()
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
        val_loss, rmse, nmse, nmse_db = evaluate(model, val_loader, criterion)
        dt = time.time() - t0

        # Track best values
        if val_loss < best_loss:
            best_loss = val_loss
            best_epoch_loss = ep

        if nmse < best_nmse:
            best_nmse = nmse
            best_epoch_nmse = ep

        if rmse < best_rmse:
            best_rmse = rmse
            best_epoch_rmse = ep

        if nmse_db < best_nmse_db:
            best_nmse_db = nmse_db
            best_epoch_nmse_db = ep

        # Logging
        if ep % show_every == 0:
            print(f"[{ep:02d}/{epochs:03d}] "
                  f"TrainLoss: {train_loss:.6f}  "
                  f"ValLoss: {val_loss:.6f}  "
                  f"Val RMSE: {rmse:.4f}  "
                  f"Val NMSE: {nmse:.4f}  "
                  f"Val NMSE_dB: {nmse_db:.6f} dB  "
                  f"TrainTime: {dt:.2f}s")

    # Summary of best metrics
    print("=" * 60)
    print(f"=> Best ValLoss   : {best_loss:.6f}   (epoch {best_epoch_loss})")
    print(f"=> Best Val NMSE  : {best_nmse:.4f}   (epoch {best_epoch_nmse})")
    print(f"=> Best Val RMSE  : {best_rmse:.4f}   (epoch {best_epoch_rmse})")
    print(f"=> Best Val NMSE_dB: {best_nmse_db:.4f} dB (epoch {best_epoch_nmse_db})")
    print("=" * 60)

    return model


In [66]:
import sys, os, contextlib

# --- Tee that mirrors stdout to both console and a file ---
class Tee:
    def __init__(self, filename, mode="w", encoding="utf-8"):
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        self.file = open(filename, mode, encoding=encoding)
        self.stdout = sys.stdout

    def write(self, data):
        self.file.write(data)
        self.stdout.write(data)

    def flush(self):
        self.file.flush()
        self.stdout.flush()

    def close(self):
        self.file.close()

    # context manager support
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc, tb):
        self.close()
        # don't suppress exceptions
        return False

# ----------------------------------------------------------
# 
base_dir = "/home/dlghdbs200/LWM_denver"

print("=== MLP Training ===")
mlp = MLP(hidden=256, depth=3, pdrop=0.1)
mlp_path = os.path.join(base_dir, "MLP.txt")
with Tee(mlp_path, "w") as tee:
    with contextlib.redirect_stdout(tee):
        print("=== Training MLP ===")
        run(mlp, epochs=50, lr=2e-3)
print(f"\nMLP training log saved to {mlp_path}")

print("\n=== CNN Training ===")
cnn = CNN1D()
cnn_path = os.path.join(base_dir, "CNN.txt")
with Tee(cnn_path, "w") as tee:
    with contextlib.redirect_stdout(tee):
        print("=== Training CNN ===")
        run(cnn, epochs=50, lr=1e-3)
print(f"\nCNN training log saved to {cnn_path}")


=== MLP Training ===
=== Training MLP ===
[01/050] TrainLoss: 0.004440  ValLoss: 0.000197  Val RMSE: 0.0140  Val NMSE: 0.0040  Val NMSE_dB: -24.013113 dB  TrainTime: 1.83s
[02/050] TrainLoss: 0.001046  ValLoss: 0.000141  Val RMSE: 0.0119  Val NMSE: 0.0028  Val NMSE_dB: -25.468136 dB  TrainTime: 2.05s
[03/050] TrainLoss: 0.000912  ValLoss: 0.000131  Val RMSE: 0.0114  Val NMSE: 0.0026  Val NMSE_dB: -25.802987 dB  TrainTime: 1.75s
[04/050] TrainLoss: 0.000850  ValLoss: 0.000134  Val RMSE: 0.0116  Val NMSE: 0.0027  Val NMSE_dB: -25.685398 dB  TrainTime: 1.88s
[05/050] TrainLoss: 0.000816  ValLoss: 0.000160  Val RMSE: 0.0127  Val NMSE: 0.0032  Val NMSE_dB: -24.907831 dB  TrainTime: 1.80s
[06/050] TrainLoss: 0.000786  ValLoss: 0.000194  Val RMSE: 0.0139  Val NMSE: 0.0039  Val NMSE_dB: -24.075181 dB  TrainTime: 1.84s
[07/050] TrainLoss: 0.000774  ValLoss: 0.000085  Val RMSE: 0.0092  Val NMSE: 0.0017  Val NMSE_dB: -27.647259 dB  TrainTime: 1.86s
[08/050] TrainLoss: 0.000741  ValLoss: 0.000108 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x76fc571ddae0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


[11/050] TrainLoss: 0.000192  ValLoss: 0.000483  Val RMSE: 0.0220  Val NMSE: 0.0097  Val NMSE_dB: -20.126447 dB  TrainTime: 2.12s
[12/050] TrainLoss: 0.000160  ValLoss: 0.000191  Val RMSE: 0.0138  Val NMSE: 0.0038  Val NMSE_dB: -24.160908 dB  TrainTime: 6.43s
[13/050] TrainLoss: 0.000181  ValLoss: 0.000113  Val RMSE: 0.0106  Val NMSE: 0.0023  Val NMSE_dB: -26.428096 dB  TrainTime: 2.59s
[14/050] TrainLoss: 0.000177  ValLoss: 0.000135  Val RMSE: 0.0116  Val NMSE: 0.0027  Val NMSE_dB: -25.648187 dB  TrainTime: 1.96s
[15/050] TrainLoss: 0.000144  ValLoss: 0.000109  Val RMSE: 0.0104  Val NMSE: 0.0022  Val NMSE_dB: -26.599590 dB  TrainTime: 2.40s
[16/050] TrainLoss: 0.000148  ValLoss: 0.000347  Val RMSE: 0.0186  Val NMSE: 0.0070  Val NMSE_dB: -21.559683 dB  TrainTime: 2.26s
[17/050] TrainLoss: 0.000151  ValLoss: 0.000089  Val RMSE: 0.0095  Val NMSE: 0.0018  Val NMSE_dB: -27.449760 dB  TrainTime: 2.31s
[18/050] TrainLoss: 0.000132  ValLoss: 0.000079  Val RMSE: 0.0089  Val NMSE: 0.0016  Val N